In [89]:
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import statistics as st


In [90]:
data = pd.read_json('day.json')

In [ ]:
data.columns= ['date', 'open', 'high', 'low', 'close', 'volume']

print(data.head())

In [ ]:
#Calculating moving averages for 50 & 200 days
data['MA50'] = data['close'].rolling(51).mean()
data['MA200'] = data['close'].rolling(201).mean()

print(data.head(60))

In [93]:
#Creating buy/sell at entry exit positions

def buy_sell(data):
    signalbuy = []
    signalsell = []
    position = False

    for i in range(len(data)):
        if data['MA50'][i] > data['MA200'][i]:
            if position == False :
                signalbuy.append(data['close'][i])
                signalsell.append(np.nan)
                position = True
            else:
                signalbuy.append(np.nan)
                signalsell.append(np.nan)
        elif data['MA50'][i] < data['MA200'][i]:
            if position == True:
                signalbuy.append(np.nan)
                signalsell.append(data['close'][i])
                position = False
            else:
                signalbuy.append(np.nan)
                signalsell.append(np.nan)
        else:
            signalbuy.append(np.nan)
            signalsell.append(np.nan)
    return pd.Series([signalbuy, signalsell])


In [ ]:
data['buy_signal_price'], data['sell_signal_price'] = buy_sell(data)
print(data.head(1000))

In [ ]:
#Merge buy and signal positions
data['buy_sell'] = data[['buy_signal_price','sell_signal_price']].sum(1)

pd.set_option('display.max_rows', 500)
print(data.head(600))

In [ ]:
#Calculating percentage change on buy sell
data['pct_change'] = data['buy_sell'].pct_change()
print(data.head(600))

In [ ]:
#Total return

initial_investment = float(10000)
pct_sum = 5.3262
total_return = ((pct_sum*initial_investment)- initial_investment)
print(total_return)

In [214]:
data['cumulative_pct'] = data['pct_change'].cumsum()

In [ ]:
data = data.drop([4590,4591,4592,4593,4594,4595,4596,4597,4598,4599,4600,4601,4602,4603,4604])


In [ ]:
data['date1'] = data['date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S+0530'))
print(data)

In [ ]:
data.info()


In [ ]:
dateend=np.datetime64('2021-09')
datestart=np.datetime64('2003-01')

duration= dateend-datestart

print(duration)

In [ ]:
#annualize_return
annualized_return=((total_return/initial_investment)**(12/224) -1)*100
print(annualized_return)

In [ ]:
data.dtypes

In [ ]:
#sharpe ratio
risk_free_return = 4
standard_deviation = st.stdev(data['buy_sell'])
sharpe_ratio = ((annualized_return- risk_free_return)/(2*standard_deviation))*100
print(sharpe_ratio)


In [222]:
peaks = data['buy_sell'].cummax()
drawdwons = (total_return- peaks)/peaks
print(drawdwons)

0             inf
1             inf
2             inf
3             inf
4             inf
          ...    
4585    56.369049
4586    56.369049
4587    56.369049
4588    56.369049
4589    56.369049
Name: buy_sell, Length: 4590, dtype: float64


In [229]:
def MaxDrawdown(buy_sell):
    mdd = 0
    maxprev = buy_sell[0]
    for i in range(1, len(buy_sell)):
        mdd = np.fmax(mdd, (maxprev- buy_sell[i])/maxprev)
        maxprev = np.fmax(maxprev, buy_sell[i])
    return mdd
    print('mdd')